# cache

Commands and population data for importing in to a database.

Even M$ visual studio products have to cache symbols somewhere.

I suppose this notebook should offline documentation too.

Rules:

- Cache functions and caches must be idempotent, calling the function multiple times with the same or different parameters will produce enjoyable results.
- Calling functions without any parameters should result in the default cache updating procedure, run for some time, and then exit.
- Caches stored in this notebook should be small, less than 1 MB each.
- Caches should be uniquely indexed and sorted so it is easy to display when something changes.



## notebook cache

Information about the current library cached.


### TODO: keyword index

A cache of question marked sentences to use as keywords in looking up functions.

TODO: look at media server caching techniques.


### TODO: imports cache

A cache of all imports in every cell, for making project trees.

TODO: use this cache to detect when a module changes in import notebook.


#### the cache

imports cache?

This code is automatically replaced with import code and stored in this notebook.


#### TODO: create import cache?



### exports cache

A cache of all exports, no matter the language.


#### the cache

exports cache?

This code is automatically replaced with export code and stored in this notebook.


In [ ]:

// export cache automatically replaced
var exportsCache = []

module.exports = exportsCache



#### update export cache?


In [ ]:
var importer = require('../Core')
var updateCode = importer.import('update code cell')

function sortAlphaNumeric(a, b) {
    // convert to strings and force lowercase
    a = typeof a === 'string' ? a.toLowerCase() : a.toString();
    b = typeof b === 'string' ? b.toLowerCase() : b.toString();

    return a.localeCompare(b);
}

function cleanCache(exportsCache) {
    var allCells = importer.interpret()
    var cellIds = allCells.map(cell => cell.id)
    
    // clean up ids that exist in the index but not in memory
    var extra = exportsCache.filter(e => cellIds.includes(e[1]))
    extra.forEach(e => exportsCache.splice(cellIds.indexOf(e[1]), 1))
    
    exportsCache.sort((a, b) => {
        return sortAlphaNumeric(a[1], b[1])
    })
}

function updateNotebook(exports) {
    // load current cache
    var exportsCache = importer.import('exports cache')
    var exportIds = exportsCache.map(e => e[1])
    var cacheCell = importer.interpret('exports cache')
    
    // update with results from search
    exports.forEach(e => {
        var i = exportIds.indexOf(e[1])
        if(i === -1) {
            exportsCache.push(e)
        } else {
            exportsCache.splice(i, 1, e)
        }
    })
    
    cleanCache(exportsCache)
    
    var code = `
// export cache automatically replaced
var exportsCache = ${JSON.stringify(exportsCache, null, 4)}

module.exports = exportsCache
`
    updateCode(cacheCell, code)
}

module.exports = updateNotebook;


#### create export cache?

TODO: generalize this so the database can be anywhere, not just a JSON notebook

TODO: make this a demonstration of idempotence, it keeps loading files.ipynb

TODO: sort and index in an Array instead of object with properties


In [1]:
var path = require('path')
var assert = require('assert')
var importer = require('../Core')
var updateNotebook = importer.import('update export cache')
var getExports = importer.import('get exports from source')

function promiseExports(cell) {
    return new Promise(resolve => {
        setTimeout(() => {
            var result
            try {
                if (process.memoryUsage().heapUsed > 500000000) {
                    throw new Error('out of memory')
                }
                result = getExports(cell.code)
            } catch (e) {
                result = e.message
            }
            resolve([Date.now(), cell.id, result])
        }, 100)
    })
}

function createExportCache(search) {
    var exportsCache = importer.import('exports cache')
    var exportIds = exportsCache.map(e => e[1])
    var allCells = importer.interpret()
    var cellIds = allCells.map(cell => cell.id)
    var notebooks = cellIds
        .map(k => k.replace(/\.ipynb\[[0-9]+\]$/ig, '.ipynb'))
        .filter((f, i, arr) => arr.indexOf(f) === i)
    
    // add files that don't exist
    var missing = notebooks.filter(n => !exportIds.includes(`${n}[0]`))
    if(!search) {
        search = missing.pop()
    }
    
    // TODO: update any changed notebooks first
    
    // update the oldest record
    if(!search) {
        exportsCache.sort((a, b) => a[0] - b[0])
        search = exportsCache[0]
    }
    
    if(!search) {
        throw new Error('nothing to do with notebook cache')
    }
    
    var allCells = [].concat.apply([], importer.interpret(search))
    var allCellIds = allCells.map(c => c.id)
    console.log(`caching ${search} - ${allCells.length} cells`)
    // get only first occurrence
    allCells = allCells
        .filter((c, i) => allCellIds.indexOf(c.id) == i)
        .filter(c => c.code.length < 100000)

    return Promise
        .all(allCells.map(promiseExports))
        .then(updateNotebook)
}

module.exports = createExportCache

if(typeof $$ !== 'undefined') {
    createExportCache()
}


SyntaxError: Unexpected token ]

#### TODO: cache every notebook in a seperate process?

### notebook cache tools

Some tools for when to update the notebook cache.



#### TODO: stale code

Determine when to update a notebook. Handles the mtime property on cells.

TODO: depends on import cache to build a tree of dependencies


#### update code cell?

Find the code cell following the specified lookup markdown and update the contents.


In [ ]:
var fs = require('fs')

function updateCode(cell, code) {
    var notebook = JSON.parse(fs.readFileSync(cell.filename))
    var match = (/\/\/.*/ig).exec(code)[0]
    
    assert(match && match.length > 2,
           `nothing to match, include a "//" comment`)
    // replace code cell with new code
    // make sure the cell if where the cache says
    assert(notebook.cells[cell.to].source
           .join('')
           .includes(match),
           `code cell "${match}" could not be located`)
    
    notebook.cells[cell.to].source = code.split('\n')
        .map(line => line + '\n')
    
    fs.writeFileSync(cell.filename, JSON.stringify(notebook, null, 4))
}

module.exports = updateCode


## TODO: bookmark cache

Always wanted to write something like evernote to automatically pull in the content of pages I bookmark. Incognito mode on news sites to avoid payment. Ad block plugins installed.  Might as well make ad-blocked cached copies of every page I visit.




## TODO: documentation cache

Tools for searching and downloading documention as it related to this library.

